In [1]:
import qualified Data.Map as Map
import qualified Data.Set as Set
import qualified Data.Set.Ordered as OSet
import Control.Monad.Writer
import Control.Monad.Identity
import Control.Monad.State
import Control.Monad.Reader
import Control.Monad.Trans.Maybe
import Control.Applicative
import Data.List
import Data.Tuple

In [2]:
type Vertex = Int
type Cost = Int
data Edge = Edge { vertex:: Vertex, cost:: Cost } deriving (Eq, Show, Ord)

In [3]:
newtype Graph = Graph { adjacencyList :: Map.Map Vertex [Edge] } deriving (Eq, Show)

In [4]:
constructGraph :: [(Vertex, Edge)] -> Graph
constructGraph edges = Graph $ foldr (\e g -> Map.insertWith (++) (fst e) [snd e] g) Map.empty edges

In [5]:
graph = constructGraph [(0, Edge 1 2), (1, Edge 0 2), (1, Edge 2 2), (2, Edge 1 2), (2, Edge 3 3),
    (3, Edge 2 3), (1, Edge 3 2), (3, Edge 1 2), (0, Edge 3 1), (3, Edge 0 1), (0, Edge 4 1), (4, Edge 0 1)]
    
graph1 = constructGraph [(0, Edge 3 1), (0, Edge 4 1), (1, Edge 2 1), (2, Edge 1 1), (3, Edge 0 1), (3, Edge 6 1),
    (4, Edge 0 1), (5, Edge 3 1), (5, Edge 6 1), (6, Edge 3 1), (6, Edge 5 1)]

<h4>DFS and BFS</h4>

In [7]:
getNeighbors :: Vertex -> Graph -> [Vertex]
getNeighbors v g = fmap vertex (Map.findWithDefault [] v (adjacencyList g))

filterVisitedNeighbors :: [Vertex] -> Set.Set Vertex -> [Vertex]
filterVisitedNeighbors listVertices set = filter (\x -> not (Set.member x set)) listVertices

In [10]:
bfs :: Vertex -> Graph -> [[Vertex]]
bfs s g = go [s] [] g (Set.singleton s) [[s]]
    where go :: [Vertex] -> [Vertex] -> Graph -> Set.Set Vertex -> [[Vertex]] -> [[Vertex]]
          go [] [] g visited result = result
          go [] nextLevel g visited result = go nextLevel [] g visited (result ++ [nextLevel])
          go (x:xs) nextLevel g visited result = 
              go xs (nextLevel ++ unvisitedNeighbors x visited) g (Set.union visited (Set.fromList $ unvisitedNeighbors x visited)) result
          unvisitedNeighbors x = filterVisitedNeighbors (getNeighbors x g)

In [12]:
bfsWithLogging :: Vertex -> Graph -> WriterT [String] Identity [[Vertex]]
bfsWithLogging s g = go [s] [] g (Set.singleton s) [[s]]
    where go :: [Vertex] -> [Vertex] -> Graph -> Set.Set Vertex -> [[Vertex]] -> WriterT [String] Identity [[Vertex]]
          go [] [] g visited result = return result
          go [] nextLevel g visited result = go nextLevel [] g visited (result ++ [nextLevel])
          go (x:xs) nextLevel g visited result = do
              tell ["Visiting :" ++ show x]
              go xs (nextLevel ++ unvisitedNeighbors x visited) g (Set.union visited (Set.fromList $ unvisitedNeighbors x visited)) result
          unvisitedNeighbors x = filterVisitedNeighbors (getNeighbors x g)

In [13]:
runIdentity . runWriterT $ bfsWithLogging 0 graph

([[0],[1,3,4],[2]],["Visiting :0","Visiting :1","Visiting :3","Visiting :4","Visiting :2"])

In [14]:
type Env = (Graph, Set.Set Vertex)
type DFSState = ([Vertex], Set.Set Vertex)
type Stack = [Vertex]

type DFSResult = ReaderT Env (MaybeT (StateT Vertex Identity)) DFSState

In [ ]:
dfsNextVisit :: Stack -> DFSState -> Maybe ([Vertex], DFSState)
dfsNextVisit [] _ = Nothing
dfsNextVisit (x:xs) (g, visited) = if not (Set.member x visited) 
                                     then Just (x, (xs, Set.insert x visited))
                                   else dfsNextVisit xs (g, visited)                            

In [ ]:
dfs :: Vertex -> Graph -> [Vertex]
dfs s g = go [s] g (Set.empty) []
    where go :: Stack -> Graph -> Set.Set Vertex -> [Vertex] -> [Vertex]
          go [] _ _ dfsTrail = dfsTrail
          go (x:xs) g visited dfsTrail = if not (Set.member x visited)
                                           then go ((getNeighbors x g) ++ xs) g (Set.insert x visited) (dfsTrail ++ [x])
                                         else go xs g visited dfsTrail

In [16]:
type DFSState = ([Vertex], OSet.OSet Vertex)
type Env = Graph
type DFSNextState = ReaderT Env (StateT DFSState Identity) Int

In [17]:
{-# LANGUAGE FlexibleContexts #-}

dfsGetNext :: DFSNextState
dfsGetNext = do
    graph <- ask
    s <- get
    let candidates = fst s
    let visited = snd s
    let ret = case candidates of
                []     -> return 0 :: DFSNextState
                (x:xs) -> if x `OSet.member` visited 
                            then put (xs, visited) >> (return 1 :: DFSNextState)
                          else put (getNeighbors x graph ++ xs, x OSet.<| visited) >> (return 1 :: DFSNextState)
    ret

In [16]:
{-# LANGUAGE RankNTypes, KindSignatures #-}
replicateMUntil :: forall (m :: * -> *) a. Monad m => (m a -> m Bool) -> m a -> m [a]
replicateMUntil f ma = do
    res <- f ma
    if res then liftM2 (:) ma (replicateMUntil f ma) else return []

In [19]:
isNextAvailable :: DFSNextState -> ReaderT Env (StateT DFSState Identity) Bool
isNextAvailable nextState = do
    s <- get
    return (fst s /= [])

In [20]:
dfsStartState :: DFSState
dfsStartState = ([0], OSet.empty)

dfsSteps = replicateMUntil isNextAvailable dfsGetNext

runStateT (runReaderT dfsSteps graph) dfsStartState

Identity ([1,1,1,1,1,1,1,1,1,1,1,1,1],([],fromList [4,3,2,1,0]))

In [21]:
runStateT (runReaderT dfsSteps graph1) dfsStartState

Identity ([1,1,1,1,1,1,1,1,1,1],([],fromList [4,5,6,3,0]))

<h4>Topological Sort</h4>

In [6]:
-- Let us define a more general graph, which can have any type of Vertex, rather than just enforcing it to be Int
data GeneralEdge a b = GeneralEdge { general_vertex:: a, general_cost:: b } deriving (Eq, Show, Ord)

data GeneralGraph a b = GeneralGraph { adjacencyList :: Map.Map a [GeneralEdge a b], 
                            incomingDegree :: Map.Map a Int } deriving (Eq, Show)

In [7]:
addDirectedEdges (a, e) (m, m') = (Map.insertWith (++) a [e] m, 
                                    Map.insertWith (+) (general_vertex e) 1 (Map.insertWith (+) a 0 m'))

constructGeneralDirectedGraph :: (Ord a, Num b) => [(a, GeneralEdge a b)] -> GeneralGraph a b
constructGeneralDirectedGraph edges = uncurry GeneralGraph (foldr addDirectedEdges (Map.empty, Map.empty) edges)

In [8]:
addUndirectedEdges (a, e) (m, m') = (Map.insertWith (++) (general_vertex e) [GeneralEdge a (general_cost e)] 
                                        (Map.insertWith (++) a [e] m),
                                    Map.insertWith (+) a 1 (Map.insertWith (+) (general_vertex e) 1 m'))

constructGeneralUndirectedGraph :: (Ord a, Num b) => [(a, GeneralEdge a b)] -> GeneralGraph a b
constructGeneralUndirectedGraph edges = uncurry GeneralGraph (foldr addUndirectedEdges (Map.empty, Map.empty) edges)

In [9]:
graph' = constructGeneralDirectedGraph [(0, GeneralEdge 1 1), (0, GeneralEdge 2 1), (0, GeneralEdge 3 1)]

In [10]:
getStartVertex :: GeneralGraph a b -> Maybe a
getStartVertex g = fmap fst (find (\(k, d) -> d == 0) (Map.toList $ incomingDegree g))

In [21]:
type TSortState a = ([a], Map.Map a Int)
type GEnv a b = GeneralGraph a b
type TSortNextState a b = ReaderT (GEnv a b) (WriterT [String] (StateT (TSortState a) Identity)) (Maybe a)

In [55]:
tSortNextState :: (Show a, Ord a) => TSortNextState a b
tSortNextState = do
    graph <- ask
    (candidates, degreesMap) <- get
    case candidates of
        []     -> tell ["No more candidates!"] >> return Nothing
        (x:xs) -> tell ["Choosing candidate : " ++ show x] >>
                  put (xs ++ newVerticesWithZeroIncomingDegree, updatedDegreesMap) >>
                  return (Just x)
                    where (newVerticesWithZeroIncomingDegree, updatedDegreesMap) =
                            foldr (\(GeneralEdge v _) (c, m) -> (if m Map.! v == 1 then c ++ [v] 
                                else c, Map.insertWith (+) v (-1) m)) ([], degreesMap) 
                                    (Map.findWithDefault [] x (adjacencyList graph))

In [56]:
isNextAvailableForTSort :: Ord a => TSortNextState a b 
                    -> ReaderT (GEnv a b) (WriterT [String] (StateT (TSortState a) Identity)) Bool
isNextAvailableForTSort nextState = do
    s <- get
    return (fst s /= [])

In [57]:
topologicalSort graph = fmap (runStateT (runWriterT (runReaderT tSortSteps graph))) tSortStartState
    where tSortSteps = replicateMUntil isNextAvailableForTSort tSortNextState
          tSortStartState = fmap (\x -> ([x], incomingDegree graph)) (getStartVertex graph)

In [58]:
topologicalSort graph'

Just (Identity (([Just 0,Just 3,Just 2,Just 1],["Choosing candidate : 0","Choosing candidate : 3","Choosing candidate : 2","Choosing candidate : 1"]),([],fromList [(0,0),(1,0),(2,0),(3,0)])))

In [59]:
graph'' = constructGeneralDirectedGraph [(0, GeneralEdge 1 1), (0, GeneralEdge 2 1), (1, GeneralEdge 2 1),
        (2, GeneralEdge 3 1), (3, GeneralEdge 4 1), (3, GeneralEdge 5 1), (4, GeneralEdge 5 1)]

In [60]:
topologicalSort graph''

Just (Identity (([Just 0,Just 1,Just 2,Just 3,Just 4,Just 5],["Choosing candidate : 0","Choosing candidate : 1","Choosing candidate : 2","Choosing candidate : 3","Choosing candidate : 4","Choosing candidate : 5"]),([],fromList [(0,0),(1,0),(2,0),(3,0),(4,0),(5,0)])))